## Project: Development of a reduced pediatric injury prediction model
Created by: Thomas Hartka, MD, MS  
Date created: 12/3/20  
  
This notebook read in data from CISS, extracts necessary features and outcomes from specific tables, then combines the data into a single CSV.  

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
encode = 'iso-8859-1'

## Read in crash data from CISS and extract features
Determine if there were multiple collisions (more than one event), find highest DVTOTAL, and PDOF associated with highest DVTOTAL

In [3]:
# add CDC data
ciss_cdc17 = pd.read_csv("../Data/CISS/2017-Pilot/CDC.CSV", encoding=encode)
ciss_cdc18= pd.read_csv("../Data/CISS/2018/CDC.CSV", encoding=encode)

# set year
ciss_cdc17['YEAR'] = 2017
ciss_cdc18['YEAR'] = 2018

# combine years
ciss_cdc = ciss_cdc17.append(ciss_cdc18).reset_index()

In [4]:
# create vehicle key
ciss_cdc['VEH_KEY'] = ciss_cdc['CASENUMBER'].astype(str)  + "_" + ciss_cdc['VEHNO'].astype(str)  

# change unknown to -999
ciss_cdc['DVTOTAL'] = ciss_cdc.apply(lambda x: x['DVTOTAL'] if x['DVTOTAL']<999 else -999, axis=1)

In [5]:
# get list of unique vehicles
cdc = ciss_cdc[['VEH_KEY','CASENUMBER','VEHNO']].drop_duplicates().copy()

In [6]:
# count number of impacts
impacts = ciss_cdc.groupby('VEH_KEY').size()
impacts = pd.DataFrame({'EVENTS':impacts}).reset_index()
impacts['MULTICOLL'] = impacts.apply(lambda x: 1 if x['EVENTS'] > 1 else 0, axis=1)

# join with cdc
cdc = cdc.merge(impacts, how='left', on='VEH_KEY')

In [7]:
# find highest DVTOTAL
max_dv = ciss_cdc.groupby('VEH_KEY')['DVTOTAL'].max()
max_dv = pd.DataFrame({'DVTOTAL':max_dv}).reset_index()

# join with cdc
cdc = cdc.merge(max_dv, how='left', on='VEH_KEY')

In [8]:
# find PDOF associated with highest DVTOTAL
idx = ciss_cdc.groupby('VEH_KEY')['DVTOTAL'].transform(max) == ciss_cdc['DVTOTAL']
max_pdof = ciss_cdc[idx][['VEH_KEY','PDOF']]

# join with cdc
cdc = cdc.merge(max_pdof, how='left', on='VEH_KEY')

## Read in GV data from CISS and join to crash data

Develop rollover status, quarter turns, etc.

In [9]:
# get occupant data
ciss_gv17 = pd.read_csv("../Data/CISS/2017-Pilot/GV.CSV", encoding=encode)
ciss_gv18= pd.read_csv("../Data/CISS/2018/GV.CSV", encoding=encode)

# set year
ciss_gv17['YEAR'] = 2017
ciss_gv18['YEAR'] = 2018

ciss_gv = ciss_gv17.append(ciss_gv18).reset_index()

In [10]:
# create vehicle key
ciss_gv['VEH_KEY'] = ciss_gv['CASENUMBER'].astype(str)  + "_" + ciss_gv['VEHNO'].astype(str)  

In [11]:
# change unknown to -99 and change name to ROLL_TURNS
ciss_gv['ROLL_TURNS'] = ciss_gv.apply(lambda x: x['ROLLTURN'] if (x['ROLLTURN']<99) else -99, axis=1)

# change end-over-end to 20
ciss_gv['ROLL_TURNS'] = ciss_gv.apply(lambda x: x['ROLL_TURNS'] if (x['ROLL_TURNS']!=98) else 20, axis=1)

# create rollover flag
ciss_gv['ROLLED'] = ciss_gv.apply(lambda x: 1 if (x['ROLL_TURNS']>0) else 0, axis=1)

In [12]:
# rename speed limit to match NASS
ciss_gv = ciss_gv.rename(columns={'SPEEDLIMIT':'SPLIMIT'})

# set 999 to nan
ciss_gv['SPLIMIT'] = ciss_gv.apply(lambda x: x.SPLIMIT if (x.SPLIMIT!=999) else np.nan, axis=1)

In [13]:
# merge quarter turns with cdc
cdc = cdc.merge(ciss_gv[['VEH_KEY','ROLL_TURNS','ROLLED','SPLIMIT']], how='left', on='VEH_KEY')

## Read in occupant data from CISS and join with crash data
Store all occupant data initially.

In [14]:
# get occupant data
ciss_occ17 = pd.read_csv("../Data/CISS/2017-Pilot/OCC.CSV", encoding=encode)
ciss_occ18= pd.read_csv("../Data/CISS/2018/OCC.CSV", encoding=encode)

# set year
ciss_occ17['YEAR'] = 2017
ciss_occ18['YEAR'] = 2018

ciss = ciss_occ17.append(ciss_occ18).reset_index()

In [15]:
# create key
ciss['KEY'] = ciss["YEAR"].astype(str) + "_" + ciss["PSU"].astype(str) + "_" + ciss["CASENO"].astype(str) + "_"\
                + ciss["VEHNO"].astype(str) + "_" + ciss["OCCNO"].astype(str)

# create vehicle key
ciss['VEH_KEY'] = ciss['CASENUMBER'].astype(str)  + "_" + ciss['VEHNO'].astype(str)  

In [16]:
# join with crash data
ciss = ciss.merge(cdc.drop(columns=['CASENUMBER','VEHNO']), how='left', on='VEH_KEY')

In [17]:
ciss['ENTRAPMENT'] = ciss.apply(lambda x: 1 if (x['ENTRAP'] in [1,2,3]) else 0, axis=1)

## Read in child seat data and add to occupants

In [18]:
# get occupant data
ciss_cs17 = pd.read_csv("../Data/CISS/2017-Pilot/CHILDSEAT.CSV", encoding=encode)
ciss_cs18= pd.read_csv("../Data/CISS/2018/CHILDSEAT.CSV", encoding=encode)

# set year
ciss_cs17['YEAR'] = 2017
ciss_cs18['YEAR'] = 2018

ciss_cs = ciss_cs17.append(ciss_cs18).reset_index()

In [19]:
# remove case with unknown occupant in child seat
ciss_cs=ciss_cs[~ciss_cs.OCCNO.isna()]

In [20]:
# create key
ciss_cs['KEY'] = ciss_cs["YEAR"].astype(str) + "_" + ciss_cs["PSU"].astype(str) + "_" + ciss_cs["CASENO"].astype(str) + "_"\
                + ciss_cs["VEHNO"].astype(str) + "_" + ciss_cs["OCCNO"].astype(int).astype(str)

In [21]:
# join with crash data
ciss = ciss.merge(ciss_cs[['KEY','ORIENTATION','CHILDSEATYPE']], how='left', on='KEY')

## Determine death and death in same compartment

In [22]:
# flag if patient died
ciss['DIED'] = ciss.apply(lambda x: 1 if ((x['PARINJSEV']==4) | (x['DEATH']>0)) else 0, axis=1)

# if police report indicates patient died but we don't know when
#  then set death = 1
ciss['DEATH'] = ciss.apply(lambda x: 1 if (((x['DEATH']==0) | (x['DEATH']==99)) & (x['PARINJSEV']==4)) else x['DEATH'], axis=1)
                           
# if occupant died and iss<24, set to 24                       
ciss['ISS'] = ciss.apply(lambda x: 24 if ((x['DIED']==1) & (x['ISS']<24)) else x['ISS'], axis=1)

In [23]:
# number of occupants in each vehicle
vocc = pd.DataFrame(ciss.groupby('VEH_KEY')['KEY'].count()).rename(columns={'KEY':'VEH_OCC_NUM'})

# number of occupants that died in each vehicle
vdied = pd.DataFrame(ciss.groupby('VEH_KEY')['DIED'].sum()).rename(columns={'DIED':'VEH_DEATH'})

# merge into occupant data
ciss = ciss.merge(vocc, on='VEH_KEY', how='left')

# merge into occupant data
ciss = ciss.merge(vdied, on='VEH_KEY', how='left')

In [24]:
# set other death flag (if someone else in the vehicle died)
ciss['OTHER_DEATH'] = ciss.apply(lambda x: 1 if ((x['VEH_DEATH'] - x['DIED']) > 0) else 0, axis=1)

## Determine intrusions

In [25]:
# get occupant data
ciss_int17 = pd.read_csv("../Data/CISS/2017-Pilot/INTRUSION.CSV", encoding=encode)
ciss_int18= pd.read_csv("../Data/CISS/2018/INTRUSION.CSV", encoding=encode)

# set year
ciss_int17['YEAR'] = 2017
ciss_int18['YEAR'] = 2018

ciss_int = ciss_int17.append(ciss_int18).reset_index()

In [26]:
# create vehicle key
ciss_int['VEH_KEY'] = ciss_int['CASENUMBER'].astype(str)  + "_" + ciss_int['VEHNO'].astype(str)  

In [27]:
# if intrusion is unknown, set to 0
ciss_int['INTMAG'] = ciss_int['INTMAG'].replace(9,0)

In [28]:
# create catastrophic intrusion flag
ciss_int['INTCAT'] = ciss_int.apply(lambda x: 1 if x['INTMAG']==7 else 0, axis=1)

In [29]:
# determine max intrusion for each vehicle
intmax = pd.DataFrame(ciss_int.groupby('VEH_KEY')[['INTMAG','INTCAT']].max()).reset_index().rename(columns={'INTMAG':'INTMAX'})

# set flag for intrusions over 18 inches
intmax['INT18'] = intmax.apply(lambda x: 1 if ((x['INTMAX']>=5) & (x['INTMAX']<=8)) else 0, axis=1)

# join to ciss data
ciss = ciss.merge(intmax, on='VEH_KEY', how='left')

In [30]:
# create flag for 12 inches of intrusion at seat position
ciss_int['INT12OCC'] = ciss_int.apply(lambda x: 1 if (x['INTMAG']>=4) & (x['INTMAG']<8) else 0, axis=1)

In [31]:
# join to ciss data
ciss = ciss.merge(ciss_int[['VEH_KEY','SEATLOC','INTMAG','INT12OCC']], on=['VEH_KEY','SEATLOC'], how='left')

## Determine if any airbag deployment

In [32]:
# get occupant data
ciss_ab17 = pd.read_csv("../Data/CISS/2017-Pilot/AIRBAG.CSV", encoding=encode)
ciss_ab18= pd.read_csv("../Data/CISS/2018/AIRBAG.CSV", encoding=encode)

# set year
ciss_ab17['YEAR'] = 2017
ciss_ab18['YEAR'] = 2018

ciss_ab = ciss_ab17.append(ciss_ab18).reset_index()

In [33]:
# create vehicle key
ciss_ab['VEH_KEY'] = ciss_ab['CASENUMBER'].astype(str)  + "_" + ciss_ab['VEHNO'].astype(str)  

In [34]:
# make flag for airbag deployment
ciss_ab['ABDEPLY'] = ciss_ab.apply(lambda x: 1 if x.BAGDEPLOY == 1 else 0, axis=1)

In [35]:
# determine if any airbag deployed
airbag = ciss_ab.groupby('VEH_KEY').sum().reset_index()[['VEH_KEY','ABDEPLY']]

In [36]:
airbag['ABDEPLY'] = airbag.apply(lambda x: 1 if x.ABDEPLY>=1 else 0, axis=1)

In [37]:
# join to ciss data
ciss = ciss.merge(airbag, on=['VEH_KEY'], how='left')

# if missing in AB table set as not deployment
ciss['ABDEPLY'] = ciss.ABDEPLY.fillna(0)

## Determine Ejection

In [38]:
# get occupant data
ciss_ejt17 = pd.read_csv("../Data/CISS/2017-Pilot/EJECT.CSV", encoding=encode)
ciss_ejt18= pd.read_csv("../Data/CISS/2018/EJECT.CSV", encoding=encode)

# set year
ciss_ejt17['YEAR'] = 2017
ciss_ejt18['YEAR'] = 2018

ciss_ejt = ciss_ejt17.append(ciss_ejt18).reset_index()

In [39]:
# create key
ciss_ejt['KEY'] = ciss_ejt["YEAR"].astype(str) + "_" + ciss_ejt["PSU"].astype(str) + "_" + ciss_ejt["CASENO"].astype(str) + "_"\
                + ciss_ejt["VEHNO"].astype(str) + "_" + ciss_ejt["OCCNO"].astype(str)

In [40]:
# convert unknowns to 0
ciss_ejt['EJECTTYPE'] = ciss_ejt['EJECTTYPE'].replace(9,0)

# rename to EJECTION
ciss_ejt = ciss_ejt.rename(columns={'EJECTTYPE':'EJECTION'})

In [41]:
# create flag for partial ejection (also unknown degree)
ciss_ejt['EJECTION_PARTIAL'] = ciss_ejt.apply(lambda x: 1 if x['EJECTION']>=2 else 0, axis=1)

# create flag for complete ejection
ciss_ejt['EJECTION_COMPLETE'] = ciss_ejt.apply(lambda x: 1 if x['EJECTION']==1 else 0, axis=1)

# create flag for any ejection
ciss_ejt['EJECTION'] = ciss_ejt.apply(lambda x: 0 if x['EJECTION']==0 else 1, axis=1)

In [42]:
# join to ciss data
ciss = ciss.merge(ciss_ejt[['KEY','EJECTION','EJECTION_PARTIAL','EJECTION_COMPLETE']], on='KEY', how='left')

## Determine crash year

In [43]:
# get occupant data
ciss_gv17 = pd.read_csv("../Data/CISS/2017-Pilot/GV.CSV", encoding=encode)
ciss_gv18= pd.read_csv("../Data/CISS/2018/GV.CSV", encoding=encode)

# set year
ciss_gv17['YEAR'] = 2017
ciss_gv18['YEAR'] = 2018

ciss_gv = ciss_gv17.append(ciss_gv18).reset_index()

In [44]:
# create vehicle key
ciss_gv['VEH_KEY'] = ciss_gv['CASENUMBER'].astype(str)  + "_" + ciss_gv['VEHNO'].astype(str)  

In [45]:
# join to ciss data
ciss = ciss.merge(ciss_gv[['VEH_KEY','MODELYR']], on='VEH_KEY', how='left')

## Filter peds cases and set missing data to NA

In [46]:
# only age <=18 years
ciss = ciss[ciss['AGE']<=18]

count = len(ciss)
print("Peds occupants: ", count)

# mark sex as nan when unknown 
ciss['SEX'] = ciss.apply(lambda x: np.nan if x.SEX==9 else x.SEX, axis=1)

# mark Delta-V as nan when unknown 
ciss['DVTOTAL'] = ciss.apply(lambda x: np.nan if x.DVTOTAL==-999 else x.DVTOTAL, axis=1)

# mark PDOF as nan when unknown 
ciss['PDOF'] = ciss.apply(lambda x: np.nan if x.PDOF>360 else x.PDOF, axis=1)

# mark rollover turns as nan when unknown 
ciss['ROLL_TURNS'] = ciss.apply(lambda x: np.nan if x.ROLL_TURNS<0 else x.ROLL_TURNS, axis=1)

# mark childseat turns as nan when unknown 
ciss['CHILDSEATYPE'] = ciss.apply(lambda x: np.nan if x.CHILDSEATYPE==99 else x.CHILDSEATYPE, axis=1)

# mark belt use as nan when unknown 
ciss['BELTUSE'] = ciss.apply(lambda x: np.nan if x.BELTUSE==99 else x.BELTUSE, axis=1)

# filter unknown ISS
ciss = ciss[ciss.ISS!=99]
print("Excluded ISS: ", count-len(ciss), "count:", len(ciss))
count = len(ciss)

Peds occupants:  2406
Excluded ISS:  168 count: 2238


## Impute weight height

In [47]:
# find mean height and weight for each age
mean_measure = ciss[(ciss['WEIGHT']<999) & (ciss['HEIGHT']<999)].groupby('AGE').agg({'WEIGHT':'mean','HEIGHT':'mean'})

In [48]:
# impute missing weight
ciss['WGT_IMP'] = ciss.apply(lambda x: x['WEIGHT'] if x['WEIGHT']<999 else mean_measure.iloc[x['AGE'],0], axis=1)

# impute missing height
ciss['HGT_IMP'] = ciss.apply(lambda x: x['HEIGHT'] if x['HEIGHT']<999 else mean_measure.iloc[x['AGE'],1], axis=1)

## Determine PDOF

In [49]:
# get position of occupant
ciss['SEAT_SIDE'] = ciss['SEATLOC'].mod(10)
ciss['SEAT_ROW'] = round(ciss['SEATLOC'] / 10)

In [50]:
# separate PDOF into quadrants
ciss['PDOF_FRONT'] = ciss.apply(lambda x: 1 if ((x['PDOF']<45) | (x['PDOF']>315)) else 0, axis=1)
ciss['PDOF_RIGHT'] = ciss.apply(lambda x: 1 if ((x['PDOF']>45) & (x['PDOF']<135)) else 0, axis=1)
ciss['PDOF_LEFT'] = ciss.apply(lambda x: 1 if ((x['PDOF']>135) & (x['PDOF']<225)) else 0, axis=1)
ciss['PDOF_REAR'] = ciss.apply(lambda x: 1 if ((x['PDOF']>225) & (x['PDOF']<315)) else 0, axis=1)

In [51]:
# determine nearside or farside (middle is considered farside)
ciss['PDOF_NEARSIDE'] = ciss.apply(lambda x: 1 if (((x['PDOF_LEFT']==1) & (x['SEAT_SIDE']==1)) |\
                                                  ((x['PDOF_RIGHT']==1) & (x['SEAT_SIDE']==3))) else 0, axis=1)

ciss['PDOF_FARSIDE'] = ciss.apply(lambda x: 1 if (((x['PDOF_LEFT']==1) & (x['SEAT_SIDE']!=1)) |\
                                                  ((x['PDOF_RIGHT']==1) & (x['SEAT_SIDE']!=3))) else 0, axis=1)

## Determine proper restraint
Based on method by Doud, et al. in "Evaluation of developmental metrics for utilization in a pediatric advanced automatic crash notification algorithm", Traffic Injury Prevention Volume 17, 2016.

In [52]:
# default is properly restrained
ciss['PROP_RESTRAINT'] = 1

# children 12 years or under in front seat are improperly restrained
ciss['PROP_RESTRAINT'] = ciss.apply(lambda x: 0 if ((x['AGE']<=12) & (x['SEAT_ROW']==1)) else x['PROP_RESTRAINT'], axis=1)

# children 0-1 years should be in rear facing 
ciss['PROP_RESTRAINT'] = ciss.apply(lambda x: 0 if ((x['AGE']<=1) & (x['ORIENTATION']!=1)) else x['PROP_RESTRAINT'], axis=1)

# children 2-4 years <18kg should be forward facing
ciss['PROP_RESTRAINT'] = ciss.apply(lambda x: 0 if ((x['AGE']>=2) & (x['AGE']<=4) & (x['WGT_IMP']<18) & (x['ORIENTATION']!=2)) else x['PROP_RESTRAINT'], axis=1)

# children 2-4 years >=18kg should use a booster 
ciss['PROP_RESTRAINT'] = ciss.apply(lambda x: 0 if ((x['AGE']>=2) & (x['AGE']<=4) & (x['WGT_IMP']>=18) 
                                                    & (~np.isin(x['CHILDSEATYPE'],[4,5,6]))) else x['PROP_RESTRAINT'], axis=1)

# children 5-7 years <36 kg or <145 cm should have booster
ciss['PROP_RESTRAINT'] = ciss.apply(lambda x: 0 if ((x['AGE']>=5) & (x['AGE']<=7) & ((x['WGT_IMP']<36) | (x['HGT_IMP']<145))\
                                                    & (~np.isin(x['CHILDSEATYPE'],[4,5,6]))) else x['PROP_RESTRAINT'], axis=1)

# children 8 and above should be wearing lap and shoulder
ciss['PROP_RESTRAINT'] = ciss.apply(lambda x: 0 if ((x['AGE']>=8) & (x['BELTUSE']!=4)) else x['PROP_RESTRAINT'], axis=1)

# make flag for unrestrained
ciss['NO_RESTRAINT'] = ciss.apply(lambda x: 1 if (((x['BELTUSE']==0)|(math.isnan(x['BELTUSE']))) & \
                                                  ((x['CHILDSEATYPE']==0) |(math.isnan(x['CHILDSEATYPE'])))) else 0, axis=1)

# make flaf for any restraint
ciss['ANY_RESTRAINT'] = ciss.apply(lambda x: 0 if (x['NO_RESTRAINT']==1) else 1, axis=1)

# if unrestrainted is 1, properly restrained is 0
ciss['PROP_RESTRAINT'] = ciss.apply(lambda x: 0 if (x['NO_RESTRAINT']==1) else x['PROP_RESTRAINT'], axis=1)

## Read in Injury data and Create outcome flags

In [53]:
# get occupant data
ciss_injury17 = pd.read_csv("../Data/CISS/2017-Pilot/INJURY.CSV", encoding=encode)
ciss_injury18 = pd.read_csv("../Data/CISS/2018/INJURY.CSV", encoding=encode)

# set year
ciss_injury17['YEAR'] = 2017
ciss_injury18['YEAR'] = 2018

ciss_injury = ciss_injury17.append(ciss_injury18).reset_index()

In [54]:
# create key
ciss_injury['KEY'] = ciss_injury["YEAR"].astype(str) + "_" + ciss_injury["PSU"].astype(str) + "_" + ciss_injury["CASENO"].astype(str) + "_"\
                + ciss_injury["VEHNO"].astype(str) + "_" + ciss_injury["OCCNO"].astype(int).astype(str)

In [55]:
# ISS>=16
ciss['ISS16'] = ciss.apply(lambda x: 1 if ((x['ISS']>=16) & (x['ISS']<97)) else 0, axis=1)

# MAIS head (region 1)  
# find max head injury severity
head_injuries = ciss_injury[ciss_injury['REGION']==1].groupby('KEY')['AIS'].max()

# join with ciss
ciss = ciss.merge(head_injuries, how='left', on='KEY').rename(columns={'AIS':'MAIS_HEAD'})

# MAIS thorax (region 4)  
# find max head injury severity
thorax_injuries = ciss_injury[ciss_injury['REGION']==4].groupby('KEY')['AIS'].max()

# join with ciss
ciss = ciss.merge(thorax_injuries, how='left', on='KEY').rename(columns={'AIS':'MAIS_THORAX'})

# MAIS abd (region 5)  
# find max head injury severity
abd_injuries = ciss_injury[ciss_injury['REGION']==5].groupby('KEY')['AIS'].max()

# join with ciss
ciss = ciss.merge(abd_injuries, how='left', on='KEY').rename(columns={'AIS':'MAIS_ABD'})

In [56]:
# replace nan for MAIS with 0
ciss[['MAIS','MAIS_HEAD','MAIS_THORAX','MAIS_ABD']] = ciss[['MAIS','MAIS_HEAD','MAIS_THORAX','MAIS_ABD']].fillna(0)

## Identify occupants with target injuries
Target injury list is AIS98 and not compatible with AIS15, so set to null value (-1).

In [57]:
ciss['TARGET_INJ']=-1

## Store data

In [58]:
# select variables
ciss_output = ciss[['KEY','YEAR','AGE','SEX','WGT_IMP','HGT_IMP','PROP_RESTRAINT','ANY_RESTRAINT','ABDEPLY','SEAT_ROW','ENTRAPMENT',\
                   'DVTOTAL','SPLIMIT','MULTICOLL','PDOF_FRONT','PDOF_REAR','PDOF_NEARSIDE','PDOF_FARSIDE','ROLLED','ROLL_TURNS',\
                   'INTCAT','INT18','INT12OCC','EJECTION','EJECTION_PARTIAL','EJECTION_COMPLETE',\
                   'DIED','OTHER_DEATH','ISS','ISS16','MAIS','MAIS_HEAD','MAIS_THORAX','MAIS_ABD','TARGET_INJ','CASEWGT']].copy()

In [59]:
def float_to_int_w_nan(x):
    return x.fillna(-1).astype(int).astype(str).replace('-1',np.nan)

In [60]:
float_cols = ['SEAT_ROW','DVTOTAL','MULTICOLL','ROLLED','ROLL_TURNS','MAIS_HEAD','MAIS_THORAX','MAIS_ABD']

# change floats to integers to match NASS
for col in float_cols:
    ciss_output[col] = float_to_int_w_nan(ciss_output[col])

# covert column names to lowercase to match NASS
ciss_output.columns = ciss_output.columns.str.lower()

In [61]:
ciss_output.to_csv("../Data/CISS/CISSPeds-2017_2018-unfiltered.csv", index=False)